<a href="https://colab.research.google.com/github/junweilam/ICT3104_T20/blob/main/ICT3104_Team20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing

In [1]:
# Testing.ipynb

import ipywidgets as widgets
from IPython.display import display
import os
import shutil
import asyncio
import platform

# Create a file uploader widget
file_upload = widgets.FileUpload(
    accept='.mp4',
    multiple=False
)

# Create a button widget
save_button = widgets.Button(
    description="Save MP4",
    disabled=False,
    button_style='success'
)

def save_mp4(b):
    if file_upload.value:
        # Get the uploaded file
        if platform.system() == "Windows":
            uploaded_file = file_upload.value[0]
        elif platform.system() == "Darwin":
            uploaded_file = list(file_upload.value.values())[0]

        # Get the file name
        if platform.system() == "Windows":
            file_name = uploaded_file['metadata']['name'] if 'metadata' in uploaded_file else uploaded_file['name']

        # Specify the destination folder to save the file
        destination_folder = './test/'

        # Create the destination folder if it doesn't exist
        if not os.path.exists(destination_folder):
            os.makedirs(destination_folder)

        if platform.system() == "Windows":
            with open(os.path.join(destination_folder, file_name), 'wb') as f:
                f.write(uploaded_file['content'])
        elif platform.system() == "Darwin":
            with open(os.path.join(destination_folder, uploaded_file['metadata']['name']), 'wb') as f:
                f.write(uploaded_file['content'])

        # Disable the file uploaded and update the button text
        file_upload.disabled = True
        save_button.description = "MP4 Saved!"
    else:
        print("Please upload an MP4 file first")

display(file_upload, save_button)

save_button.on_click(save_mp4)

FileUpload(value={}, accept='.mp4', description='Upload')

Button(button_style='success', description='Save MP4', style=ButtonStyle())

In [2]:
# Testing.ipynb

if file_upload.value == 1:
    print("one")
else:
    print("zero")

zero


In [3]:
# Testing .ipynb

from IPython.display import Video

# List videos
print("Input Videos:")
for file in os.listdir("./input"):
    if file.endswith(".mp4"):
        print(file)

print("\nOutput Videos:")
for file in os.listdir("./output"):
    if file.endswith(".mp4"):
        print(file)

# TODO add dropdown to allow selection of video
selectedVideo = "input/car-turn.mp4"

# Display selected video
Video(selectedVideo)

Input Videos:


FileNotFoundError: ignored



```
# This is formatted as code
```

# Testing Checkpoint

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import os
import shutil
import asyncio

# Create a file uploader widget
file_upload = widgets.FileUpload(
    accept='.mp4',
    multiple=False
)

# Create a button widget
save_button = widgets.Button(
    description="Save MP4",
    disabled=False,
    button_style='success'
)

def save_mp4(b):
    if file_upload.value:
        # Get the uploaded file
        uploaded_file = file_upload.value[0]

        # Get the file name
        file_name = uploaded_file['metadata']['name'] if 'metadata' in uploaded_file else uploaded_file['name']

        # Specify the destination folder to save the file
        destination_folder = './test/'

        # Create the destination folder if it doesn't exist
        if not os.path.exists(destination_folder):
            os.makedirs(destination_folder)

        with open(os.path.join(destination_folder, file_name), 'wb') as f:
            f.write(uploaded_file['content'])

        # Disable the file uploaded and update the button text
        file_upload.disabled = True
        save_button.description = "MP4 Saved!"
    else:
        print("Please upload an MP4 file first")

display(file_upload, save_button)

save_button.on_click(save_mp4)

In [ ]:
if file_upload.value == 1:
    print("one")
else:
    print("zero")

In [ ]:
from IPython.display import Video

# List videos
print("Input Videos:")
for file in os.listdir("./input"):
    if file.endswith(".mp4"):
        print(file)

print("\nOutput Videos:")
for file in os.listdir("./output"):
    if file.endswith(".mp4"):
        print(file)

# TODO add dropdown to allow selection of video
selectedVideo = "input/car-turn.mp4"

# Display selected video
Video(selectedVideo)

/bin/bash: line 1: nvidia-smi: command not found
